# [Link to FMP apis](https://site.financialmodelingprep.com/developer/docs)

In [23]:
import requests
import pandas as pd
import numpy as np
from IPython.display import display

# Setting an API Key

In [1]:
api_key = "lDg6Ns4sehr6zVXCVAW2PWApFzdGZkdf"

# Setting the endpoints

In [62]:
ENDPOINTS = {
    'STOCK_LIST': 'https://financialmodelingprep.com/api/v3/stock/list',
    'COMPANY_PROFILE': 'https://financialmodelingprep.com/api/v3/profile',
    'HISTORICAL_DIVIDEND_YIELD': 'https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend',
}

def get_request_endpoint(endpoint_type: str, **kwargs):
    endpoint: str = ENDPOINTS[endpoint_type]

    if endpoint_type != 'STOCK_LIST':
        symbol: str = kwargs['symbol']
        endpoint = endpoint + f'/{symbol}'

    return endpoint + f'?apikey={api_key}'

# Retrieve the data

## Stock List

In [43]:
sl_res = requests.get(get_request_endpoint('STOCK_LIST'))

In [44]:
sl = sl_res.json()

In [45]:
sl_df = pd.DataFrame(sl)

In [46]:
sl_df.head()

,symbol,name,price,exchange,exchangeShortName,type
0,PMGOLD.AX,Perth Mint Gold,17.940,Australian Securities Exchange,ASX,etf
1,ORNAV.HE,Orion Oyj,44.600,Helsinki,HEL,stock
2,SEDANA.ST,Sedana Medical AB (publ),12.400,Stockholm Stock Exchange,STO,stock
3,KOJAMO.HE,Kojamo Oyj,9.075,Helsinki,HEL,stock
4,PUUILO.HE,Puuilo Oyj,9.230,Helsinki,HEL,stock


### Filter stock list

In [57]:
def filter_stocks(df: pd.DataFrame, stock_symbols: list):
    rows: list = []
    
    for symbol in stock_symbols:
        rows.append(df[df['symbol'] == symbol])

    filtered_df = pd.concat(rows)

    return filtered_df

In [58]:
sl_stocks = sl_df[sl_df['type'] == 'stock']

In [59]:
target_stock_symbols = ['NVDA', 'VZ', 'MO', 'ENB', 'GOOGL', 'META', 'AAPL', 'BNS', 'IBM', 'KO']

In [60]:
filtered_sl_stocks = filter_stocks(sl_stocks, target_stock_symbols)

In [61]:
filtered_sl_stocks

,symbol,name,price,exchange,exchangeShortName,type
45783,NVDA,NVIDIA Corporation,139.335,NASDAQ Global Select,NASDAQ,stock
36503,VZ,Verizon Communications Inc.,41.280,New York Stock Exchange,NYSE,stock
36450,MO,"Altria Group, Inc.",50.500,New York Stock Exchange,NYSE,stock
38208,ENB,Enbridge Inc.,40.590,New York Stock Exchange,NYSE,stock
40069,GOOGL,Alphabet Inc.,174.460,NASDAQ Global Select,NASDAQ,stock
38522,META,"Meta Platforms, Inc.",591.800,NASDAQ Global Select,NASDAQ,stock
40049,AAPL,Apple Inc.,230.100,NASDAQ Global Select,NASDAQ,stock
37056,BNS,The Bank of Nova Scotia,52.250,New York Stock Exchange,NYSE,stock
36513,IBM,International Business Machines Corporation,204.900,New York Stock Exchange,NYSE,stock
36525,KO,The Coca-Cola Company,65.920,New York Stock Exchange,NYSE,stock


## Company profiles

In [77]:
def retrieve_company_profiles(df: pd.DataFrame):
    profiles = []
    
    for i, row in df.iterrows():
        endpoint: str = get_request_endpoint('COMPANY_PROFILE', symbol=row['symbol'])

        res = requests.get(endpoint)
        body = res.json()
        profiles.append(body[0])

    return profiles

In [78]:
company_profiles = retrieve_company_profiles(filtered_sl_stocks)

In [80]:
cp_df = pd.DataFrame(company_profiles)

In [81]:
cp_df

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,NVDA,132.9400,1.669,308335656,3261018200000,0.04,42.894-144.42,-6.3950,NVIDIA Corporation,USD,...,95051,80.54906,58.785940,https://images.financialmodelingprep.com/symbo...,1999-01-22,False,False,True,False,False
1,VZ,41.8000,0.411,17747423,175962534000,2.71,35.31-45.36,0.5200,Verizon Communications Inc.,USD,...,10036,-22.62783,63.907831,https://images.financialmodelingprep.com/symbo...,1983-11-21,False,False,True,False,False
2,MO,53.9100,0.684,7750628,91982320200,4.08,39.25-54.95,3.4100,"Altria Group, Inc.",USD,...,23230,-18.73839,69.238388,https://images.financialmodelingprep.com/symbo...,1985-07-01,False,False,True,False,False
3,ENB,40.6111,0.904,3582265,88427421362,2.71,32.65-42.16,0.0211,Enbridge Inc.,USD,...,T2P 3L8,32.86762,7.722381,https://images.financialmodelingprep.com/symbo...,1984-03-15,False,False,True,False,False
4,GOOGL,175.1100,1.038,25121332,2165107102564,0.80,126.93-191.75,0.6500,Alphabet Inc.,USD,...,94043,-30.01554,204.475541,https://images.financialmodelingprep.com/symbo...,2004-08-19,False,False,True,False,False
5,META,576.4938,1.216,12632925,1458201930089,2.00,308.33-602.95,-15.3062,"Meta Platforms, Inc.",USD,...,94025,120.51516,471.284840,https://images.financialmodelingprep.com/symbo...,2012-05-18,False,False,True,False,False
6,AAPL,228.7100,1.239,50328964,3477329711000,1.00,164.08-237.49,-1.3900,Apple Inc.,USD,...,95014,84.77657,145.323426,https://images.financialmodelingprep.com/symbo...,1980-12-12,False,False,True,False,False
7,BNS,51.6450,0.974,1661698,63501142650,3.14,41.46-55.12,-0.6050,The Bank of Nova Scotia,USD,...,B3J 1W1,100.28092,-48.030916,https://images.financialmodelingprep.com/symbo...,2002-06-07,False,False,True,False,False
8,IBM,205.2600,0.697,3884264,189074838480,6.68,144.84-237.37,0.3600,International Business Machines Corporation,USD,...,10504,-58.76055,263.660552,https://images.financialmodelingprep.com/symbo...,1915-09-24,False,False,True,False,False
9,KO,65.9150,0.608,14057306,283948637000,1.94,56.06-73.53,-0.0050,The Coca-Cola Company,USD,...,30313,-40.45983,106.379827,https://images.financialmodelingprep.com/symbo...,1919-09-05,False,False,True,False,False


## Historical dividend yields

In [83]:
def retrieve_historical_dividiend_yields(df: pd.DataFrame):
    yield_data = []
    
    for i, row in df.iterrows():
        endpoint: str = get_request_endpoint('HISTORICAL_DIVIDEND_YIELD', symbol=row['symbol'])

        res = requests.get(endpoint)
        body = res.json()
        yield_data.append(body)

    return yield_data

In [84]:
historical_dividend_yields = retrieve_historical_dividiend_yields(filtered_sl_stocks)

In [85]:
historical_dividend_yields[:1]

[{'symbol': 'NVDA',
  'historical': [{'date': '2024-09-12',
    'label': 'September 12, 24',
    'adjDividend': 0.01,
    'dividend': 0.01,
    'recordDate': '2024-09-12',
    'paymentDate': '2024-10-03',
    'declarationDate': '2024-08-28'},
   {'date': '2024-06-11',
    'label': 'June 11, 24',
    'adjDividend': 0.01,
    'dividend': 0.01,
    'recordDate': '2024-06-11',
    'paymentDate': '2024-06-28',
    'declarationDate': '2024-05-22'},
   {'date': '2024-03-05',
    'label': 'March 05, 24',
    'adjDividend': 0.004,
    'dividend': 0.4,
    'recordDate': '2024-03-06',
    'paymentDate': '2024-03-27',
    'declarationDate': '2024-02-21'},
   {'date': '2023-12-05',
    'label': 'December 05, 23',
    'adjDividend': 0.004,
    'dividend': 0.4,
    'recordDate': '2023-12-06',
    'paymentDate': '2023-12-28',
    'declarationDate': '2023-11-21'},
   {'date': '2023-09-06',
    'label': 'September 06, 23',
    'adjDividend': 0.004,
    'dividend': 0.4,
    'recordDate': '2023-09-07',
  